# Mesogens with NP | Equilibrium

### Date: 12/03/2019 | System P = 1.8, Expected value of $T_c$ : 7.09 | 8 times little system

In [1]:
from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define relevant variables
p_max = 1.8;
t_max = 6.2;
copies = 2;
steps_run = 1e4;
init_file = "T_CM&NP_" + str(t_max) + "_P_" + str(p_max) + "_ramp.gsd"

In [3]:
#-----Define a simulation context

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

HOOMD-blue 2.7.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/02/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


In [4]:
#-----Extract the configuration of the system and expand the system

snap = hoomd.data.gsd_snapshot(init_file, frame = -1);
snap.replicate(copies,copies,copies);
system = hoomd.init.read_snapshot(snap);

notice(2): Group "all" created containing 8032 particles


In [5]:
#-----Define each mesogen in the local reference frame of each center of mass
rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
               types = ['A']*8,
               positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                            (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

In [6]:
#-----Declare molecules as rigid bodies
rigid.create_bodies();

notice(2): constrain.rigid(): Creating 8000 rigid bodies (adding 64000 particles)


In [7]:
#-----Define the potential energy
nl = hoomd.md.nlist.tree();
lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl);
lj.set_params(mode = 'shift')

-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----


In [8]:
#------Define the interaction
lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M', 'M', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A', 'A', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M', 'A', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP', 'M', epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP', 'A', epsilon = 1.0, sigma = 3.0);

In [9]:
#------Select an standar integrator
hoomd.md.integrate.mode_standard(dt = 0.005);

#-----Define some groups and make their union

nanoparticles = hoomd.group.type(name = 'NPs', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = 'NP_Mes', a = nanoparticles, b = mesogens);

notice(2): Group "NPs" created containing 32 particles
notice(2): Group "rigid_center" created containing 8000 particles
notice(2): Group "NP_Mes" created containing 8032 particles


In [10]:
#-----Integrate using NPT

npt = hoomd.md. integrate.npt(group = groupNP_mes, kT = t_max, tau = 3.0, tauP = 3.0, P = p_max);

In [11]:
#-----Save data

log_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_ramp.gsd"
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_ramp.log"

log = hoomd.analyze.log(filename = log_file,
                       quantities = ['num_particles', 
                                    'ndof',
                                    'translational_ndof',
                                    'rotational_ndof',
                                    'potential_energy',
                                    'kinetic_energy',
                                    'translational_kinetic_energy',
                                    'rotational_kinetic_energy',
                                    'temperature',
                                    'pressure',
                                    'volume'],
                       period = 1e2,
                       overwrite = True);
gsd = hoomd.dump.gsd(gsd_file, period = 1e2, group = hoomd.group.all(), overwrite = True);
meso_gsd = hoomd.dump.gsd(meso_gsd_file, period=1e2, group = mesogens, overwrite = True);

In [12]:
#-----Run the simulation

hoomd.run(steps_run)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 72032
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 13 / 10000 | TPS 1.2916 | ETA 02:08:52
Time 00:00:20 | Step 36 / 10000 | TPS 2.25989 | ETA 01:13:29
Time 00:00:31 | Step 58 / 10000 | TPS 2.06249 | ETA 01:20:20
Time 00:00:41 | Step 80 / 10000 | TPS 2.18902 | ETA 01:15:31
Time 00:00:51 | Step 102 / 10000 | TPS 2.18432 | ETA 01:15:31
Time 00:01:01 | Step 126 / 10000 | TPS 2.37519 | ETA 01:09:17
Time 00:01:11 | Step 149 / 10000 | TPS 2.29852 | ETA 01:11:25
Time 00:01:21 | Step 170 / 10000 | TPS 1.99813 | ETA 01:21:59
Time 00:01:32 | Step 192 / 10000 | TPS 2.05039 | ETA 01:19:43
Time 00:01:43 | Step 213 / 10000 | TPS 1.99828 | ETA 01:21:37
Time 00:01:53 | Step 233 / 10000 | TPS 1.98553 | ETA 01:21:59
Time 00:02:03 | Step 253 / 10000 | TPS 1.95151 | ETA 01:23:14
Time 00:02:13 | S

In [13]:
#-----Get volume and density information.
system.box.get_volume()

117136.68960261966

In [14]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x7fbe98029b00>),
             ('particles', <hoomd.data.particle_data at 0x7fbe98136ac8>),
             ('number_density', 0.6149396934842956),
             ('bonds', <hoomd.data.bond_data at 0x7fbe98136b38>),
             ('angles', <hoomd.data.angle_data at 0x7fbe98136ba8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x7fbe98136c18>),
             ('impropers', <hoomd.data.dihedral_data at 0x7fbe98136c88>),
             ('constraints', <hoomd.data.constraint_data at 0x7fbe98136cf8>),
             ('pairs', <hoomd.data.bond_data at 0x7fbe98136d68>),
             ('timestep', 10000)])